In [51]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm.auto import tqdm
import pandas as pd

In [2]:
sample_link = "https://leetcode.com/problems/two-sum/"
driver = webdriver.Chrome("/home/sathya/Desktop/chromedriver")
driver.get(sample_link)
time.sleep(10)
main_content = driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div/div/div[1]/div/div/div/div[2]/div/div/div[3]/div")
qn = ""
for p in main_content.find_elements_by_tag_name("p"):
    p_content = BeautifulSoup(p.text, "lxml")
    if "Example" in p_content.text:
        break
    qn += p_content.text

print(f"Scrapped qn: {qn}")


sample_input_output = BeautifulSoup(main_content.find_element_by_tag_name("pre").text, "lxml").text

if "Explanation" in sample_input_output:
    idx = sample_input_output.find("Explanation")

sample_input_output = sample_input_output[:idx]

print(f"Scrapped input output: {sample_input_output}")

tags = []

a_divs = driver.find_element_by_xpath("/html/body/div[1]/div/div/div/div/div/div[1]/div/div/div/div[2]/div/div/div[7]/div/div[2]")

for a_div in a_divs.find_elements_by_tag_name("a"):
    tags.append(a_div.get_attribute("href").split("/")[-2])

print(f"Scrapped tags: {tags}")

Scrapped qn: Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.You may assume that each input would have exactly one solution, and you may not use the same element twice.You can return the answer in any order.
Scrapped input output: Input: nums = [2,7,11,15], target = 9
Output: [0,1]

Scrapped tags: ['array', 'hash-table']


In [4]:
num_pages = 55

page_link = "https://leetcode.com/problemset/all/?page="
problem_links = []

for page_num in tqdm(range(1, num_pages+1)):
    driver = webdriver.Chrome("/home/sathya/Desktop/chromedriver")
    driver.get(page_link+str(page_num))
    time.sleep(10)
    qn_list_table = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div[1]/div[5]/div[2]/div/div")
    
    for link in qn_list_table.find_elements_by_tag_name("a"):
        href = link.get_attribute("href")
        if "solution" not in href:
            problem_links.append(href)
            # print(href)
    print(len(problem_links))
    driver.quit()

  0%|          | 0/55 [00:00<?, ?it/s]

52
102
152
202
252
302
352
402
452
502
552
602
652
702
752
802
852
902
954
1004
1054
1104
1154
1204
1253
1303
1353
1403
1453
1503
1555
1605
1655
1705
1755
1805
1855
1905
1955
2005
2055
2105
2157
2207
2257
2309
2359
2409
2459
2509
2559
2609
2659
2709
2747


In [6]:
print(f"Scrapped links count: {len(problem_links)}")
problem_links = list(set(problem_links))
print(f"Removed duplicates after count: {len(problem_links)}")

Scrapped links count: 2747
Removed duplicates after count: 2537


In [43]:
question_texts = []
input_outputs = []
tags = []
premium_count = 0

In [44]:
for i in tqdm(range(len(problem_links))):
    driver = webdriver.Chrome("/home/sathya/Desktop/chromedriver")
    driver.get(problem_links[i])
    time.sleep(5)
    
    body_content = driver.find_element_by_tag_name("body")
    
    try:
        cp_content = body_content.find_element_by_class_name("_1l1MA")
        cp_text = cp_content.text

        qn_end_index = cp_text.find("Example") 
        io_start_index = cp_text.find("Input")
        io_end_index = cp_text.find("Explanation")

        question_texts.append(cp_text[:qn_end_index])
        input_outputs.append(cp_text[io_start_index: io_end_index])

        tags_a = body_content.find_elements_by_xpath('//a[contains(@href, "tag")]')
        temp_tags = []
        for a_ in tags_a:
            temp_tags.append(a_.get_attribute("href").split("/")[-2])
        tags.append(temp_tags)
    except:
        premium_count += 1
    
    if (i+1)%50==0:
        print(len(question_texts), len(input_outputs), len(tags), premium_count)
    
    driver.quit()

  0%|          | 0/2537 [00:00<?, ?it/s]

35 35 35 15
79 79 79 21
118 118 118 32
161 161 161 39
201 201 201 49
243 243 243 57
286 286 286 64
324 324 324 76
359 359 359 91
400 400 400 100
440 440 440 110
481 481 481 119
522 522 522 128
561 561 561 139
606 606 606 144
645 645 645 155
689 689 689 161
732 732 732 168
770 770 770 180
811 811 811 189
854 854 854 196
892 892 892 208
933 933 933 217
975 975 975 225
1019 1019 1019 231
1054 1054 1054 246
1091 1091 1091 259
1131 1131 1131 269
1168 1168 1168 282
1206 1206 1206 294
1246 1246 1246 304
1287 1287 1287 313
1330 1330 1330 320
1368 1368 1368 332
1405 1405 1405 345
1443 1443 1443 357
1485 1485 1485 365
1528 1528 1528 372
1571 1571 1571 379
1614 1614 1614 386
1651 1651 1651 399
1692 1692 1692 408
1728 1728 1728 422
1772 1772 1772 428
1810 1810 1810 440
1852 1852 1852 448
1889 1889 1889 461
1931 1931 1931 469
1970 1970 1970 480
2006 2006 2006 494


In [45]:
print(f"Number of qts: {len(question_texts)}")
print(f"Number of ios: {len(input_outputs)}")
print(f"Number of tags: {len(tags)}")

Number of qts: 2035
Number of ios: 2035
Number of tags: 2035


In [50]:
df = pd.DataFrame(data={"question_text": question_texts, "input_outputs": input_outputs, "tags": tags})
df.to_csv("../datasets/leetcode.csv")